In [1]:
import pickle
from timeit import default_timer as timer

import os
import platform
import tensorflow as tf
import keras;

# *print ("Current file path: {}".format(os.path.dirname(os.path.realpath(__file__))))
print ("Current Working Directory: {}".format(os.getcwd()))
print("Python: {}, TensorFlow:{}, Keras:{}".\
      format(platform.python_version(), tf.__version__, keras.__version__))

random_seed = 1



Using TensorFlow backend.


Current Working Directory: /home/endre/git/finance_ml/src
Python: 3.6.1, TensorFlow:1.2.1, Keras:2.0.5


# Load pickled Features, Labels and Name of each Feature-array

In [2]:
print ("De-pickling features, labels and feature/label names..")
load_start = timer()

# pickled = pickle.load(open("RangeNamesFeaturesAndLabels-mk1.pickle", "rb"))
# pickled = pickle.load(open("RangeNamesFeaturesAndLabels-smallSet-6stocks.pickle", "rb"))
# pickled = pickle.load(open("RangeNamesFeaturesAndLabels-smallSet-50stocks.pickle", "rb"))
pickled = pickle.load(open("RangeNamesFeaturesAndLabels-mediumset-unknownNumStocks.pickle", "rb"))

load_millis = (timer()-load_start) * 1000
print ("De-pickle took {} ms".format(load_millis))

# {'rangeNames': rangeNames, 'features': features, 'labels': labels}
rangeNames = pickled['rangeNames']
features = pickled['features']
labels = pickled['labels']
print("\nRangeNames/features/labels len: {}/{}/{}".\
      format(len(rangeNames), len(features), len(labels)))
numFeatures = len(features[0])
print("\nNumber of Features for each feature-array: {}".format(numFeatures))
print("Number of Labels for each label-array: {}".format(len(labels[0])))

De-pickling features, labels and feature/label names..
De-pickle took 25464.75458200439 ms

RangeNames/features/labels len: 1008922/1008922/1008922

Number of Features for each feature-array: 54
Number of Labels for each label-array: 4


# Split set into Train and Test

In [3]:
import resource
import gc
def mem():
    print('Memory usage: %2.2f MB' % round(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0,1))
    
mem()
trainRangeNames = []
trainFeatures = []
trainLabels = []

testRangeNames = []
testFeatures = []
testLabels = []
mem()

gc.collect()
mem()

length = len(rangeNames)
for i in range(length):
    names = rangeNames[i]
    if (names[1] < '2016-01-01'):
        trainRangeNames.append(names)
        trainFeatures.append(features[i])
        trainLabels.append(labels[i])
    else:
        testRangeNames.append(names)
        testFeatures.append(features[i])
        testLabels.append(labels[i])

lenTrainFeatures = len(trainFeatures)

print("TRAIN RangeNames/features/labels len: {}/{}/{} - {:.2f}% of total".\
      format(len(trainRangeNames), lenTrainFeatures, len(trainLabels), (lenTrainFeatures / len(features)) * 100 ))
print("TEST RangeNames/features/labels len: {}/{}/{} - {:.2f}% of total".\
      format(len(testRangeNames), len(testFeatures), len(testLabels), (len(testFeatures) / len(features)) * 100 ))

print("\nTotal loaded features: {}, trainFeatures + testFeatures:{} - {:.4f}%".\
      format(len(features), lenTrainFeatures + len(testFeatures), ((lenTrainFeatures + len(testFeatures)) / len(features)) * 100))

Memory usage: 7899.90 MB
Memory usage: 7899.90 MB
Memory usage: 7899.90 MB
TRAIN RangeNames/features/labels len: 939830/939830/939830 - 93.15% of total
TEST RangeNames/features/labels len: 69092/69092/69092 - 6.85% of total

Total loaded features: 1008922, trainFeatures + testFeatures:1008922 - 100.0000%


In [4]:
# One-hot the labels

# 0:  5 days
# 1: 10 days
# 2: 15 days
# 3: 20 days
label_to_use = 3

good = [1,0]
bad =  [0,1]

trainLabels_onehot = [good if x[label_to_use] > 0 else bad  for x in trainLabels]
testLabels_onehot = [good if x[label_to_use] > 0 else bad  for x in testLabels]
print("One-hotted ok.")

sum_trainLabels = [sum(i) for i in zip(*trainLabels_onehot)]
sum_testLabels = [sum(i) for i in zip(*testLabels_onehot)]
print("Train labels [good, bad]: {} -> [{:.2f}%, {:.2f}%]".\
      format(sum_trainLabels, (sum_trainLabels[0] / len(trainLabels)) * 100, (sum_trainLabels[1] / len(trainLabels)) * 100))
print("Test labels  [good, bad]: {} -> [{:.2f}%, {:.2f}%]".\
      format(sum_testLabels, (sum_testLabels[0] / len(testLabels)) * 100, (sum_testLabels[1] / len(testLabels)) * 100))

One-hotted ok.
Train labels [good, bad]: [505724, 434106] -> [53.81%, 46.19%]
Test labels  [good, bad]: [39999, 29093] -> [57.89%, 42.11%]


# !! RUN FROM HERE WHEN ONLY TWEAKING NETWORK !!

In [5]:
from sklearn.utils import shuffle

currentTrainPos = 0
shuff_trainFeatures = []
shuff_trainLabels_onehot = []
def getTrainMiniBatch(miniBatchSize):
    global currentTrainPos
    global shuff_trainFeatures, shuff_trainLabels_onehot
    if (currentTrainPos + miniBatchSize > lenTrainFeatures):
        currentTrainPos = 0
    if (currentTrainPos == 0):
        print("Shuffling training features and labels..")
        shuff_trainFeatures, shuff_trainLabels_onehot = shuffle(trainFeatures, trainLabels_onehot, random_state=random_seed)
    start = currentTrainPos
    end = currentTrainPos + miniBatchSize
    currentTrainPos = end
    
    return shuff_trainFeatures[start:end], shuff_trainLabels_onehot[start:end]

# Set up Neural Network

In [6]:
n1=1200
n2=1800
n3=1200
n4=600
out=2
epochs = 5 # Not explicitly used, as we count mini_batches
mini_batch_size = 256
learning_rate = 0.0001  # 0.0001 is good, but maybe use higher learning rate when dropout'ing..
keep_prob_in_train = 0.8
keep_prob_hid_train = 0.5
transfer_function = tf.nn.elu
l2_regularization_beta = 0.025

#========
mini_batches_per_epoch = int(lenTrainFeatures / mini_batch_size)
total_mini_batches = mini_batches_per_epoch * epochs

print("Mini batches per epoch: trainFeats {} / mini_batch_size {} = {}".format(lenTrainFeatures, mini_batch_size, mini_batches_per_epoch))
print("Total mini batches: mini_batches_per_epoch {} * epochs {} = {}".format(mini_batches_per_epoch, epochs, total_mini_batches))

# Reset the default graph, so as to chuck out existing variables
tf.reset_default_graph()

with tf.variable_scope("input"):
    X = tf.placeholder(tf.float32, shape=[None, numFeatures], name="X")
    keep_prob_in = tf.placeholder(tf.float32, name="keep_prob")
    X_drop = tf.nn.dropout(X, keep_prob_in)
    labels = tf.placeholder(tf.float32, shape=[None, out], name="labels")

with tf.variable_scope("hidden_layers_common"):
    keep_prob_hid = tf.placeholder(tf.float32)

def weight_variable(name, shape):
    # return tf.get_variable(name, initializer=tf.glorot_uniform_initializer(seed=random_seed), shape=shape)
    return tf.get_variable(name, initializer=tf.contrib.layers.xavier_initializer(), shape=shape)
    
def bias_variable(name, shape):
    # return tf.get_variable(name, initializer=tf.glorot_uniform_initializer(seed=random_seed), shape=shape)
    return tf.get_variable(name, initializer=tf.contrib.layers.xavier_initializer(), shape=shape)

with tf.variable_scope("layer1"):
    w1 = weight_variable("weight", [numFeatures, n1])
    b1 = bias_variable("bias", [n1])
    l1 = tf.matmul(X_drop, w1) + b1
    l1 = transfer_function(l1)
    l1 = tf.nn.dropout(l1, keep_prob_hid)

with tf.variable_scope("layer2"):
    w2 = weight_variable("weight", [n1, n2])
    b2 = bias_variable("bias", [n2])
    l2 = tf.matmul(l1, w2) + b2
    l2 = transfer_function(l2)
    l2 = tf.nn.dropout(l2, keep_prob_hid)

with tf.variable_scope("layer3"):
    w3 = weight_variable("weight", [n2, n3])
    b3 = bias_variable("bias", [n3])
    l3 = tf.matmul(l2, w3) + b3
    l3 = transfer_function(l3)
    l3 = tf.nn.dropout(l3, keep_prob_hid)

with tf.variable_scope("layer4"):
    w4 = weight_variable("weight", [n3, n4])
    b4 = bias_variable("bias", [n4])
    l4 = tf.matmul(l3, w4) + b4
    l4 = transfer_function(l4)
    l4 = tf.nn.dropout(l4, keep_prob_hid)

with tf.variable_scope("output"):
    wy = weight_variable("weight", [n4, out])
    by = bias_variable("bias", [out])
    Y = tf.matmul(l4, wy) + by

# :: Loss function with L2 Regularization
l2_regularizer = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3) + \
                 tf.nn.l2_loss(w4) + tf.nn.l2_loss(wy)
# NOTE: L1 regularizer: tf.reduce_sum(tf.abs(tensor)) instead of tf.nn.l2_loss(tensor)
# Read here: https://github.com/tensorflow/models/blob/master/inception/inception/slim/losses.py
# Max-norm:
# https://stackoverflow.com/questions/37801832/how-can-i-implement-max-norm-constraints-in-an-mlp-in-tensorflow
# Hmm..: https://stackoverflow.com/questions/34934303/renormalize-weight-matrix-using-tensorflow

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=labels) +\
                      l2_regularization_beta * l2_regularizer)

# :: Optimizer/Trainer
optimizer = tf.train.AdamOptimizer(learning_rate)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

# Define Test/Evaluate: Accuracy: Fraction right predictions
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("Network set up.")

Mini batches per epoch: trainFeats 939830 / mini_batch_size 256 = 3671
Total mini batches: mini_batches_per_epoch 3671 * epochs 5 = 18355
Network set up.


# Train Neural Network

In [7]:
check_length = len(testFeatures)
check_trainFeatures, check_trainLabels_onehot = shuffle(trainFeatures, trainLabels_onehot, random_state=42)
check_trainFeatures = check_trainFeatures[:check_length]
check_trainLabels_onehot = check_trainLabels_onehot[:check_length]

sess = tf.Session()

# Initialize TensorFlow variables in Session
sess.run(tf.global_variables_initializer())

def printStats(what):
    train_cost, train_accuracy = sess.run([loss, accuracy], 
                                       feed_dict={X: check_trainFeatures,
                                                  labels: check_trainLabels_onehot,
                                                  keep_prob_in: 1.0,
                                                  keep_prob_hid: 1.0})
    test_cost, test_accuracy = sess.run([loss, accuracy], 
                                     feed_dict={X: testFeatures,
                                                labels: testLabels_onehot,
                                                keep_prob_in: 1.0,
                                                keep_prob_hid: 1.0})
    print("{}\n  TRAIN cost/acc:{:.4f}/{:.4f}%, TEST cost/acc:{:.4f}/{:.4f}%"\
          .format(what, train_cost, train_accuracy*100, test_cost, test_accuracy*100))

printStats("BEFORE Training")
print("-----------------------------------\n")

time_start = timer()
epoch = 0
for mini_batch in range(1, total_mini_batches+1):
    batch_inputs, batch_labels = getTrainMiniBatch(mini_batch_size)
    sess.run([train, accuracy], feed_dict={X: batch_inputs, 
                                           labels: batch_labels,
                                           keep_prob_in: keep_prob_in_train,
                                           keep_prob_hid: keep_prob_hid_train})
    if ((epoch == 0) and (mini_batch % 100 == 0)):
        printStats(' ... progress: In "Epoch" {} of {}, mini batch {} of {}'.format(epoch, epochs, mini_batch, total_mini_batches))
    if ((epoch > 0) and (mini_batch % 500 == 0)):
        print(' ... progress: In "Epoch" {} of {}, mini batch {} of {}'.format(epoch, epochs, mini_batch, total_mini_batches))
        
    if (mini_batch % mini_batches_per_epoch == 0):
        epoch += 1
        printStats('\nAfter "Epoch" {} of {}, mini batch {} of {}'.format(epoch, epochs, mini_batch, total_mini_batches))
        
training_time = timer()-time_start
printStats("\nFINISHED")
print("TRAINING took {} seconds.".format(training_time))

BEFORE Training
  TRAIN cost/acc:49.3260/47.1791%, TEST cost/acc:49.3647/44.4465%
-----------------------------------

Shuffling training features and labels..
 ... progress: In "Epoch" 0 of 5, mini batch 100 of 18355
  TRAIN cost/acc:46.7166/53.7313%, TEST cost/acc:46.6467/56.8836%
 ... progress: In "Epoch" 0 of 5, mini batch 200 of 18355
  TRAIN cost/acc:44.9110/53.9599%, TEST cost/acc:44.8659/55.8849%
 ... progress: In "Epoch" 0 of 5, mini batch 300 of 18355
  TRAIN cost/acc:43.2519/53.8485%, TEST cost/acc:43.2087/57.1716%
 ... progress: In "Epoch" 0 of 5, mini batch 400 of 18355
  TRAIN cost/acc:41.4962/53.8123%, TEST cost/acc:41.4658/56.9603%
 ... progress: In "Epoch" 0 of 5, mini batch 500 of 18355
  TRAIN cost/acc:39.7761/53.9223%, TEST cost/acc:39.7386/57.6709%
 ... progress: In "Epoch" 0 of 5, mini batch 600 of 18355
  TRAIN cost/acc:37.9926/53.5822%, TEST cost/acc:37.9717/54.3623%
 ... progress: In "Epoch" 0 of 5, mini batch 700 of 18355
  TRAIN cost/acc:36.2387/53.9527%, TES